# Quantitative Momentum Strategy

## Importing libraries

In [ ]:
import sys
!{sys.executable} -m pip install numpy
!{sys.executable} -m pip install pandas
!{sys.executable} -m pip install python-dotenv
!{sys.executable} -m pip install requests
!{sys.executable} -m pip install scipy
!{sys.executable} -m pip install xlsxwriter

In [ ]:
from dotenv import load_dotenv
import math
import numpy
import os
import pandas
import requests
from scipy.stats import percentileofscore as score
from statistics import mean
import xlsxwriter

## Importing List of Stocks

In [ ]:
stocks =  pandas.read_csv('sp_500_stocks.csv')

## Acquiring API Token

In [ ]:
load_dotenv()
key = os.getenv("ACCESS_KEY")

## Making first API call

In [ ]:
symbol = 'AAPL'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/stats/?token={key}'
data = requests.get(api_url).json()

## Parsing API result

In [ ]:
data['year1ChangePercent']

## Execution Batch API call

In [ ]:
def chuncks(lst, n):
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

In [ ]:
symbol_groups = list(chuncks(stocks['Ticker'], 100))

In [ ]:
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))

In [ ]:
data_columns = ['Ticker', 'Price', 'One Year Price Return', 'Number of Shares to Buy']
resultDataFrame = pandas.DataFrame(columns = data_columns)

In [ ]:
for symbol_string in symbol_strings[:1]:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=stats,price&token={key}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        resultDataFrame = resultDataFrame.append(
            pandas.Series([
                symbol,
                data[symbol]['price'],
                data[symbol]['stats']['year1ChangePercent'],
                'N/A'
            ], index= data_columns),
            ignore_index=True
        )

resultDataFrame

## Removing Low-momentum Stocks

In [ ]:
resultDataFrame.sort_values('One Year Price Return', ascending=False, inplace=True)
resultDataFrame = resultDataFrame[:50]
resultDataFrame.reset_index(inplace=True)
resultDataFrame

## Calculating the Number of Shares to Buy

In [ ]:
def portfolio_input():
    global portfolio_size
    try:
        portfolio_size = float(input('Enter the size of your portfolio:'))
    except ValueError:
        print('Should be a number')
        portfolio_size = float(input('Enter the size of your portfolio:'))

In [ ]:
portfolio_input()

In [ ]:
position_size = float(portfolio_size / len(resultDataFrame.index))

In [ ]:
for i in range(0, len(resultDataFrame.index)):
    price = resultDataFrame.loc[i, 'Price']
    resultDataFrame.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / price)

resultDataFrame

## Building a Better (and More Realistic) Momentum Strategy

- **High Quality** show "slow and steady" outperformance
- **Low Quality** might not show any momentum for a long time, and surge upwards

In [ ]:
hqm_columns = [
    'Ticker',
    'Price',
    'Number of Shares to Buy',
    'One Year Price Return',
    'One Year Return Percentile',
    'Six Months Price Return',
    'Six Months Return Percentile',
    'Three Months Price Return',
    'Three Months Return Percentile',
    'One Month Price Return',
    'One Month Return Percentile',
    'HQM Score'
]

hqm_dataframe = pandas.DataFrame(columns=hqm_columns)

for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=stats,price&token={key}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        hqm_dataframe = hqm_dataframe.append(
            pandas.Series([
                symbol,
                data[symbol]['price'],
                'N/A',
                data[symbol]['stats']['year1ChangePercent'],
                'N/A (Percentile)',
                data[symbol]['stats']['month6ChangePercent'],
                'N/A (Percentile)',
                data[symbol]['stats']['month3ChangePercent'],
                'N/A (Percentile)',
                data[symbol]['stats']['month1ChangePercent'],
                'N/A (Percentile)',
                'N/A'
            ], index= hqm_columns),
            ignore_index=True
        )

hqm_dataframe


### Calculating momentum percetile

In [ ]:
time_periods = [
    'One Year',
    'Six Months',
    'Three Months',
    'One Month',
]

for row in hqm_dataframe.index:
    for time_period in time_periods:
        
        change_col = f'{time_period} Price Return'

        if hqm_dataframe.loc[row, change_col] == None:
            hqm_dataframe.loc[row, change_col] = 0

for row in hqm_dataframe.index:
    for time_period in time_periods:
        
        percentile_col = f'{time_period} Return Percentile'
        change_col = f'{time_period} Price Return'

        time_period_score = score(hqm_dataframe[change_col], hqm_dataframe.loc[row, change_col])
        hqm_dataframe.loc[row, percentile_col] = time_period_score / 100

hqm_dataframe

 ### Calculating HQM Score

In [ ]:
for row in hqm_dataframe.index:
    momentum_percentiles = []
    for time_period in time_periods:
        momentum_percentiles.append(hqm_dataframe.loc[row, f'{time_period} Return Percentile'])
    hqm_dataframe.loc[row, 'HQM Score'] = mean(momentum_percentiles)

hqm_dataframe

### Selecting the 50 Best Momentum Stocks

In [ ]:
hqm_dataframe.sort_values('HQM Score', ascending=False, inplace=True)
hqm_dataframe = hqm_dataframe[:50]
hqm_dataframe

### Calculating the Number of Shares to Buy

In [ ]:
portfolio_input()

position_size = portfolio_size / len(hqm_dataframe.index)
for i in hqm_dataframe.index:
    hqm_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / hqm_dataframe.loc[i, 'Price'])

hqm_dataframe

## Write to Excel

In [ ]:
writer = pandas.ExcelWriter('recommended momentum strategy trades.xlsx', engine = 'xlsxwriter')
hqm_dataframe.to_excel(writer, 'Recommended Trades', index = False)

background_color = '#0E0A23'
font_color = '#FFFFFF'

string_format = writer.book.add_format({
    'font_color': font_color,
    'bg_color': background_color,
    'border': 1
})

dollar_format = writer.book.add_format({
    'num_format': '$ 0.00',
    'font_color': font_color,
    'bg_color': background_color,
    'border': 1
})

int_format = writer.book.add_format({
    'num_format': '0',
    'font_color': font_color,
    'bg_color': background_color,
    'border': 1
})

percent_format = writer.book.add_format({
    'num_format': '0.0%',
    'font_color': font_color,
    'bg_color': background_color,
    'border': 1
})

column_formats = {
    'A': ['Ticker',string_format],
    'B': ['Price',dollar_format],
    'C': ['Number of Shares to Buy',int_format],
    'D': ['One Year Price Return',percent_format],
    'E': ['One Year Return Percentile',percent_format],
    'F': ['Six Months Price Return',percent_format],
    'G': ['Six Months Return Percentile',percent_format],
    'H': ['Three Months Price Return',percent_format],
    'I': ['Three Months Return Percentile',percent_format],
    'J': ['One Month Price Return',percent_format],
    'K': ['One Month Return Percentile',percent_format],
    'L': ['HQM Score',percent_format]
}

for column in column_formats.keys():
    writer.sheets['Recommended Trades'].set_column(f'{column}:{column}', 18, column_formats[column][1])
    writer.sheets['Recommended Trades'].write(f'{column}1', column_formats[column][0], string_format)

writer.save()
